In [5]:
pip install selenium

     |████████████████████████████████| 904 kB 3.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import re
from urllib.request import urlopen
import schedule
import time
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
import os
from openpyxl import load_workbook


def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('d:/temp/test.xlsx', df)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2', 
                           index=False, startrow=25)

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    # Excel file doesn't exist - saving and exiting
    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [3]:
urls = ['https://www.amazon.com/1917-Sam-Mendes/dp/B082PQKLL2/','https://www.amazon.com/Abominable-Chloe-Bennet/dp/B07Y4JN2TM/','https://www.amazon.com/Despicable-Me-3-Steve-Carell/dp/B072XZXYTR/','https://www.amazon.com/Downton-Abbey-Movie-2019-Bonneville/dp/B07XVFMQK4/','https://www.amazon.com/Fast-Furious-8-Movie-Collection-Diesel/dp/B072J4CK4R/','https://www.amazon.com/Fifty-Shades-Collection-Dakota-Johnson/dp/B07B64T2XJ/','https://www.amazon.com/Green-Book-Viggo-Mortensen/dp/B07KBTKCFZ/','https://www.amazon.com/Halloween-Jamie-Lee-Curtis/dp/B07J3GTYC1/','https://www.amazon.com/Happy-Death-Day-Jessica-Rothe/dp/B07NBD2MMF/','https://www.amazon.com/Fast-Furious-Presents-Hobbs-Shaw/dp/B07VGTYC9G/','https://www.amazon.com/How-Train-Your-Dragon-Collection/dp/B07NNMNK9F','https://www.amazon.com/Jason-Bourne-Matt-Damon/dp/B01JAQEQOK/','https://www.amazon.com/Jurassic-World-Chris-Pratt/dp/B00NYC5II0/','https://www.amazon.com/Jurassic-World-Collection-Sam-Neill/dp/B07FQ6SQ76/','https://www.amazon.com/Jurassic-World-Kingdom-Chris-Pratt/dp/B07DQMJBHS/','https://www.amazon.com/Mamma-Mia-Here-We-Again/dp/B07F9GVFK6/','https://www.amazon.com/Minions-Sandra-Bullock/dp/B00NYC73JW/','https://www.amazon.com/Pitch-Perfect-Trilogy-Anna-Kendrick/dp/B07984YN58/','https://www.amazon.com/Sing-Matthew-McConaughey/dp/B01LTI0P24/','https://www.amazon.com/Fate-Furious-Vin-Diesel/dp/B06Y3MYS83/','https://www.amazon.com/First-Purge-Ylan-Noel/dp/B07F3YWCSR/','https://www.amazon.com/Illumination-Presents-Dr-Seuss-Grinch/dp/B07JYR54B7/','https://www.amazon.com/Secret-Life-Pets-Louis-C-K/dp/B01I5JTBRU/','https://www.amazon.com/Secret-Life-Pets-2/dp/B07RQ3STKG/','https://www.amazon.com/Us-Lupita-Nyongo/dp/B07PRNRTXX/','https://www.amazon.com/Invisible-Man-Leigh-Whannell/dp/B084Z4JQNM/','https://www.amazon.com/Emma-Autumn-Wilde/dp/B084QLBNVB/','https://www.amazon.com/Trolls-World-Tour-Anna-Kendrick/dp/B086PPHNJB/','https://www.amazon.com/Hunt-Ike-Barinholtz/dp/B085HNCCB7/','https://www.amazon.com/Trolls-World-Tour-2-Movie-Collection/dp/B086PLB57C/','https://www.amazon.com/Yesterday-Himesh-Patel/dp/B07T4RYGLX/','https://www.amazon.com/Illumination-Presents-2-Movie-Pack-Secret/dp/B079ZV4HCD/','https://www.amazon.com/Freaky-Vince-Vaughn/dp/B08MSGQNNC/','https://www.amazon.com/Croods-New-Age-Nicolas-Cage/dp/B08NM8T7YM/','https://www.amazon.com/All-My-Life-Marc-Meyers/dp/B08P8SJ8HV/','https://www.amazon.com/Half-Brothers-Luke-Greenfield/dp/B08P495CDJ/','https://www.amazon.com/Promising-Young-Woman-Carey-Mulligan/dp/B08QWBXYW8/','https://www.amazon.com/News-World-DVD-Tom-Hanks/dp/B08QRB3F1N/','https://www.amazon.com/Let-Him-Go-Diane-Lane/dp/B08M2BC5B5/','https://www.amazon.com/Dogs-Journey-Purpose-2-Movie-Collection/dp/B07TQYRTFR/','https://www.amazon.com/Dolittle-Robert-Downey-Jr/dp/B083N4KV3T/','https://www.amazon.com/Neighbors-Sorority-Rising-Seth-Rogen/dp/B01FY2UMOM/','https://www.amazon.com/House-Clock-Casper-Double-Feature/dp/B08CPLDRQG/','https://www.amazon.com/Glass-James-McAvoy/dp/B07MF43VWD/','https://www.amazon.com/Mortal-Engines-Hugo-Weaving/dp/6317640157/','https://www.amazon.com/Pitch-Perfect-3-Anna-Kendrick/dp/B0788XV93X/','https://www.amazon.com/Boss-Baby-Alec-Baldwin/dp/B07BX9HC7Z/','https://www.amazon.com/Shrek-Ultimate-Collection-Mike-Myers/dp/B07WKV2799/','https://www.amazon.com/Skyscraper-Dwayne-Rock-Johnson/dp/B07F5CPBPQ/','https://www.amazon.com/Nobody/dp/B0912X671V/','https://www.amazon.com/Boogie-DVD-Taylor-Takahashi/dp/B08XRXQ817/']
alltitle = pd.read_csv("/Users/a206668344/Desktop/NBCU/titleall.csv")
alltitle = alltitle.drop(columns =['Seller'])

others = []
for url in urls:
    others.append(url+'ref=olp_aod_redir_impl1?_encoding=UTF8&aod=1')
alltitle.insert(1, "Others", others, True)

In [4]:
schedule.clear()

In [5]:
def getresult():
    
    global seller
    global title
    
    print("I'm working...")
    
    titlelist = []
    sellerlist = []
    seller = []
    title = []
    

    for url in urls:
        driver = webdriver.Chrome(executable_path='/Users/a206668344/Desktop/NBCU/chromedriver2')
        driver.get(url)
        try:
            seller = driver.find_elements_by_xpath('//*[@id="tabular-buybox-truncate-1"]/span[2]/span')
            title = driver.find_elements(By.ID,"productTitle")
        except:
            pass

        for i in range(len(seller)):
            sellerlist.append(seller[i].text)
        for i in range(len(seller)):
            titlelist.append(title[i].text)
    driver.close()
    
    d = {'Title':titlelist, 'Seller':sellerlist}
    df = pd.DataFrame(d)
    
    result = alltitle.join(df.set_index('Title'), on='Title')
    result.loc[result.Seller =='Amazon.com', 'Digit'] = 1
    result.loc[result.Seller =='GRUV', 'Digit'] = 1 
    result.loc[result.Seller.isnull() == True, 'Digit'] = 0
    result.loc[result.Digit.isnull() == True, 'Digit'] = 0
    
    filename = r'/Users/a206668344/Desktop/NBCU/output.xlsx'
    sheettime = datetime.strftime(datetime.now(),'%Y-%m-%d_%H-%M')
    append_df_to_excel(filename, result, sheet_name=sheettime,index=False)
    print("I finished geting the sellers")
    
schedule.every(5).minutes.do(getresult)

while True:
    schedule.run_pending()

I'm working...


KeyboardInterrupt: 